# Scale population and employment for Postcode Sectors and Workplace Zones

In outline:
- get area definitions for Local Authority Districts (LAD - 2011 and 2016), Workplace Zones (WZ), Postcode Sectors (PS), Postcodes (PCD) in Great Britain
- read 2011 census data for workplace population at LAD, WZ
- read 2011 census data for usual resident population at LAD, PS, PCD
- convert 2011 LAD codes to 2016 equivalents
- read population and employment scenarios and baseline projection at LAD scale
- scale future employment to workplace zones
- scale future population to postcode sectors

## Citations

England, Northern Ireland, Scotland and Wales 2011 Census
- Office for National Statistics ; National Records of Scotland ; Northern Ireland Statistics and Research Agency (2017): 2011 Census aggregate data. UK Data Service (Edition: February 2017). DOI: http://dx.doi.org/10.5257/census/aggregate-2011-2
This information is licensed under the terms of the Open Government Licence [http://www.nationalarchives.gov.uk/doc/open-government-licence/version/2].

England and Wales 2001 Census
- Office for National Statistics (2011): 2001 Census aggregate data (Edition: May 2011). UK Data Service. DOI: http://dx.doi.org/10.5257/census/aggregate-2001-2


In [ ]:
import os
import zipfile
from glob import glob

import geopandas as gpd
import pandas as pd
import requests
from geopandas.tools import explicit_crs_from_epsg
from shapely.geometry import Point

## Get boundaries and lookups

In [ ]:
def download(url, filename, dirname=".", force=False):
    if force or not os.path.exists(filename):
        r = requests.get(url, stream=True)
        with open(filename, 'wb') as fd:
            for chunk in r.iter_content(chunk_size=128):
                fd.write(chunk)
    if filename.endswith(".zip"):
        with zipfile.ZipFile(filename,"r") as zf:
            zf.extractall(dirname)    

In [ ]:
# Scotland Workplace Zones
# ref: https://www.nrscotland.gov.uk/statistics-and-data/geography/our-products/census-datasets/2011-census/2011-boundaries
url = "https://www.nrscotland.gov.uk/files//geography/products/workplacezones2011scotland.zip"
download(url, "data/workplacezones2011scotland.zip")
wz_sc = gpd.read_file("data/WorkplaceZones2011Scotland/WorkplaceZones2011Scotland.shp").rename(columns={
    'LADCD': 'lad11cd',
    'WZCD': 'wz11cd'
})
wz_sc.head()

In [ ]:
# England and Wales Workplace Zones
# ref: https://geoportal.statistics.gov.uk/datasets/workplace-zones-december-2011-full-clipped-boundaries-in-england-and-wales
url = "https://opendata.arcgis.com/datasets/a399c2a5922a4beaa080de63c0a218a3_0.zip?outSR=%7B%22latestWkid%22%3A27700%2C%22wkid%22%3A27700%7D"
download(url, "data/workplacezones2011ew.zip", "data")
wz_ew = gpd.read_file("data/Workplace_Zones_December_2011_Full_Clipped_Boundaries_in_England_and_Wales.shp")[
    ['lad11cd', 'wz11cd', 'geometry']
]
wz_ew.head()

In [ ]:
# GB Postcode Sectors
# ref: https://datashare.is.ed.ac.uk/handle/10283/2597
url = "https://datashare.is.ed.ac.uk/bitstream/handle/10283/2597/GB_Postcodes.zip?sequence=1&isAllowed=y"
download(url, "data/gb_postcodes.zip", "data")
ps = gpd.read_file("data/GB_Postcodes/PostalSector.shp").rename(columns={
    'StrSect': 'postcode_sector'
})[
    ['postcode_sector', 'geometry']
]
ps.head()

In [ ]:
# GB Postcodes
# ref: https://www.ordnancesurvey.co.uk/business-and-government/products/code-point-open.html
# Request download manually (email/link system)
pcd_headers = pd.read_csv("data/codepo_gb/Doc/Code-Point_Open_Column_Headers.csv", header=1).rename(columns={
    'Admin_district_code': 'lad11cd',
    'Postcode': 'postcode',
    'Eastings': 'eastings',
    'Northings': 'northings'
})
pcd_headers

In [ ]:
pcd_dfs = []
for fname in glob("data/codepo_gb/Data/CSV/*.csv"):
    df = pd.read_csv(fname, header=None, names=pcd_headers.columns)[
        ['postcode', 'lad11cd', 'eastings', 'northings']
    ]
    pcd_dfs.append(df)
pcds = pd.concat(pcd_dfs)

In [ ]:
pcds = gpd.GeoDataFrame(
    pcds, 
    geometry=gpd.points_from_xy(pcds.eastings, pcds.northings), 
    crs=27700
)

In [ ]:
pcds.postcode = pcds.postcode.apply(lambda d: d.replace(' ', ''))
pcds.head()

In [ ]:
len(pcds)

## Get 2011 census data 
- population by Postcode Sector, LAD
- workplace population by Workplace Zone, LAD

In [ ]:
def unique_geo_len_sum(df):
    return len(df.GEOGRAPHY_CODE.unique()) == len(df), len(df), df.OBS_VALUE.sum()

### Postcode resident population
Postcode data sourced from Nomis and Scotland Census

In [ ]:
# ref https://www.nomisweb.co.uk/census/2011/postcode_headcounts_and_household_estimates
url = 'https://www.nomisweb.co.uk/output/census/2011/Postcode_Estimates_Table_1.csv'
download(url, "data/Postcode_Estimates_Table_1.csv", "data")

# ref https://www.scotlandscensus.gov.uk/bulletin-figures-and-tables
url = 'https://www.scotlandscensus.gov.uk/documents/censusresults/release1c/rel1c2tableA1.csv'
download(url, "data/rel1c2tableA1.csv", "data")

In [ ]:
pcd_pop_ew = pd.read_csv("data/Postcode_Estimates_Table_1.csv").rename(columns={
    'Postcode': 'postcode',
    'Total': 'pop11',
    'Occupied_Households': 'hh11'
})[
    ['postcode', 'pop11', 'hh11']
]
pcd_pop_ew.postcode = pcd_pop_ew.postcode.apply(lambda d: d.replace(' ', ''))
pcd_pop_ew.head()

In [ ]:
pcd_pop_sc = pd.read_csv("data/rel1c2tableA1.csv", header=2).rename(columns={
    '2011 Postcode': 'postcode',
    'Usually resident population': 'pop11',
    'Households with usual residents': 'hh11'
}).dropna()
pcd_pop_sc.postcode = pcd_pop_sc.postcode.apply(lambda d: str(d).replace(' ', ''))
pcd_pop_sc.pop11 = pcd_pop_sc.pop11.astype('int')
pcd_pop_sc.hh11 = pcd_pop_sc.hh11.astype('int')
pcd_pop_sc.head()

In [ ]:
pcd_pop = pd.concat([pcd_pop_ew, pcd_pop_sc], axis=0).sort_values('postcode')

In [ ]:
len(pcd_pop), pcd_pop.pop11.sum()

In [ ]:
pcd_pop11 = pd.read_csv("data/pc_pop_2011.csv").sort_values('postcode')
pcd_pop11.head()

In [ ]:
len(pcd_pop11.oslaua.unique())

### Workplace zone workplace population
Workplace population density is available by workplace zones through NOMIS
https://www.nomisweb.co.uk/census/2011/wp102ew

In [ ]:
# Scotland
url = 'http://www.scotlandscensus.gov.uk/documents/additional_tables/WP103SCwz.csv'
download(url, "data/WP103SCwz.csv", "data")

# England and Wales - ref https://www.nomisweb.co.uk/census/2011/wp102ew
# Download via javascript

In [ ]:
unique_geo_len_sum(wp_wz11_ew)

## Scale and project scenarios